In [1]:
import pandas as pd
import numpy as np

In [2]:
team_hitter = []
for i in  range(5):
    team_hitter.append(pd.read_csv(f'2020빅콘테스트_스포츠투아이_제공데이터_팀타자_20{16+i}.csv'))
hitter_data = pd.concat([team_hitter[i] for i in range(len(team_hitter))])

In [3]:
team_pitcher = []
for i in  range(5):
    team_pitcher.append(pd.read_csv(f'2020빅콘테스트_스포츠투아이_제공데이터_팀투수_20{16+i}.csv'))
pitcher_data = pd.concat([team_pitcher[i] for i in range(len(team_hitter))])

In [4]:
hitter = []
for i in  range(5):
    hitter.append(pd.read_csv(f'2020빅콘테스트_스포츠투아이_제공데이터_개인타자_20{16+i}.csv', encoding="cp949"))
hitter_p_data = pd.concat([hitter[i] for i in range(len(hitter))])

In [5]:
pitcher = []
for i in  range(5):
    pitcher.append(pd.read_csv(f'2020빅콘테스트_스포츠투아이_제공데이터_개인투수_20{16+i}.csv', encoding="cp949"))
pitcher_p_data = pd.concat([pitcher[i] for i in range(len(pitcher))])

In [6]:
team = []
for i in  range(5):
    team.append(pd.read_csv(f'2020빅콘테스트_스포츠투아이_제공데이터_등록선수_20{16+i}.csv', encoding="cp949"))
team_data = pd.concat([team[i] for i in range(len(hitter))])

In [7]:
game = []
for i in  range(5):
    game.append(pd.read_csv(f'2020빅콘테스트_스포츠투아이_제공데이터_경기_20{16+i}.csv', encoding="cp949"))
game_data = pd.concat([game[i] for i in range(len(hitter))])

In [8]:
# #데이터 추가
# hitter_data["strike_out_rate"] = hitter_data["KK"] / hitter_data["AB"]
# hitter_data["4ball_rate"] = 
# hitter_data["ERA_mean"] = 
# batting_average = hitter_data.groupby("T_ID")["RUN"].sum() / hitter_data.groupby("T_ID")["AB"].sum() 

In [9]:
team_hitter

[               G_ID   GDAY_DS T_ID VS_T_ID  HEADER_NO TB_SC  PA  AB  RBI  RUN  \
 0     20160401HHLG0  20160401   LG      HH          0     B  47  42    4    5   
 1     20160401HHLG0  20160401   HH      LG          0     T  52  46    4    4   
 2     20160401HTNC0  20160401   NC      HT          0     B  36  30    5    5   
 3     20160401HTNC0  20160401   HT      NC          0     T  38  34    3    4   
 4     20160401KTSK0  20160401   SK      KT          0     B  36  36    4    4   
 ...             ...       ...  ...     ...        ...   ...  ..  ..  ...  ...   
 1435  20161008SSSK0  20161008   SS      SK          0     T  39  36    5    6   
 1436  20161009KTNC0  20161009   NC      KT          0     B  41  39    3    4   
 1437  20161009KTNC0  20161009   KT      NC          0     T  43  38    5    7   
 1438  20161009WOLT0  20161009   LT      WO          0     B  38  36    8    8   
 1439  20161009WOLT0  20161009   WO      LT          0     T  38  33    5    5   
 
       ...  BB

### 볼넷 비율은 볼넷 합/타수로 나눠주기때문에 four_ball_average 수정 
### 안타당 타점을 얻을 비율 Avg_point에 넣음
### 한선수가 noting 타석에 섰는데 안타도 X 희타, 희비 X , 4구, 고의4구, 사구도 아닐확률

In [10]:
four_ball_average = hitter_p_data.groupby('P_ID')["BB"].sum() / hitter_p_data.groupby('P_ID')['PA'].sum() #선수의 타석당 볼넷 갯수의 평균
steal = hitter_p_data.groupby('P_ID')["SB"].sum() / (hitter_p_data.groupby('P_ID')["SB"].sum()+hitter_p_data.groupby('P_ID')["CS"].sum()) #타자당 도루 성공확률 구하기
RBI = hitter_p_data.groupby('P_ID')["RBI"].sum() / hitter_p_data.groupby('P_ID')["PA"].sum() #선수 타석당 평균 타점
Avg_point = (hitter_p_data.groupby('P_ID')["RBI"].sum() +hitter_p_data.groupby('P_ID')["RBI"].sum()) / hitter_p_data.groupby('P_ID')["HIT"].sum() #안타당 평균 타점
nothing = 1-(hitter_p_data.groupby('P_ID')["HIT"].sum() + hitter_p_data.groupby('P_ID')["SH"].sum()+hitter_p_data.groupby('P_ID')["SF"].sum() + hitter_p_data.groupby('P_ID')["BB"].sum() + hitter_p_data.groupby('P_ID')["IB"].sum() +hitter_p_data.groupby('P_ID')["HP"].sum())/ hitter_p_data.groupby('P_ID')["PA"].sum() #안타당 평균 타점 #타석에 나가서 아무것도 못하고 돌아올 확률

In [11]:
for_team = pd.DataFrame(four_ball_average) # team_data에 merge할 데이터들 모음
for_team = pd.concat([for_team,steal,RBI, Avg_point, nothing],axis=1) # 포볼비율, 도루, 타점

for_team.columns = ["four_ball","steal","RBI", "Avg_point", "noting"]
team_data = pd.merge(team_data,for_team, on="P_ID",how="left")


team_data = team_data.dropna(axis=0)#결측치는 비 출전 선수인듯 제거
team_data

GDAY_DS T_ID   P_ID ENTRY_YN  four_ball     steal       RBI  \
0       20160401   HH  60404        Y   0.057692  0.000000  0.019231   
7       20160401   HH  61700        N   0.116348  0.533333  0.075110   
8       20160401   HH  62700        Y   0.046997  0.537037  0.111619   
11      20160401   HH  62797        N   0.087308  0.611111  0.090542   
14      20160401   HH  63700        N   0.057377  0.000000  0.122951   
...          ...  ...    ...      ...        ...       ...       ...   
520322  20200719   LG  78217        Y   0.096016  0.738462  0.066394   
520323  20200719   LG  78361        N   0.084586  0.692308  0.092105   
520325  20200719   LG  79109        Y   0.105634  0.740000  0.119718   
520327  20200719   LG  79150        Y   0.081315  0.700000  0.081315   
520328  20200719   LG  79192        N   0.057665  0.548387  0.157525   

        Avg_point    noting  
0        0.400000  0.807692  
7        0.739130  0.649485  
8        0.879177  0.672977  
11       0.805755  0.645918  
14       1.363636  0.713115  
...           ...       ...  
520322   0.560345  0.636364  
520323   0.753846  0.644737  
520325   1.030303  0.624120  
520327   0.770492  0.676471  
520328   1.140917  0.627754  

[185333 rows x 9 columns]

# 상대 투수 방어율을 어떻게 구해야하나? 투수코드를 상대 타자수만큼 돌리면 될거같다.


In [12]:
pitcher_p_data

G_ID   GDAY_DS T_ID VS_T_ID  HEADER_NO TB_SC   P_ID  START_CK  \
0     20160401HHLG0  20160401   LG      HH          0     B  60146         0   
1     20160401HHLG0  20160401   LG      HH          0     B  61891         0   
2     20160401HHLG0  20160401   LG      HH          0     B  62146         0   
3     20160401HHLG0  20160401   LG      HH          0     B  62698         1   
4     20160401HHLG0  20160401   HH      LG          0     T  65764         0   
...             ...       ...  ...     ...        ...   ...    ...       ...   
2882  20200719WOSK0  20200719   WO      SK          0     T  67313         1   
2883  20200719WOSK0  20200719   WO      SK          0     T  67391         0   
2884  20200719WOSK0  20200719   WO      SK          0     T  68341         0   
2885  20200719WOSK0  20200719   WO      SK          0     T  69399         0   
2886  20200719WOSK0  20200719   SK      WO          0     B  75138         0   

      RELIEF_CK  CG_CK  ...  KK GD  WP  BK  ERR  R  ER  P_WHIP_RT  P2_WHIP_RT  \
0             1      0  ...   3  0   0   0    0  0   0   0.000000    1.000000   
1             1      0  ...   1  0   0   0    0  0   0   0.000000    1.800000   
2             1      0  ...   0  0   0   0    0  0   0   0.000000    3.000000   
3             0      0  ...   4  1   0   0    0  4   4   0.818182    1.588235   
4             1      0  ...   1  0   0   0    0  1   1   0.000000    1.500000   
...         ...    ...  ...  .. ..  ..  ..  ... ..  ..        ...         ...   
2882          0      0  ...   7  0   1   0    0  0   0   0.000000    1.000000   
2883          1      0  ...   1  0   0   0    0  0   0   0.000000    0.000000   
2884          1      0  ...   2  0   1   0    0  4   4   0.000000    0.000000   
2885          1      0  ...   0  0   0   0    0  0   0   3.000000    3.000000   
2886          1      0  ...   0  0   0   0    0  0   0   0.000000    0.000000   

      CB_WHIP_RT  
0           1.50  
1           1.50  
2           0.00  
3           3.75  
4           0.00  
...          ...  
2882        2.25  
2883        0.00  
2884        1.50  
2885        0.00  
2886        0.00  

[27804 rows x 38 columns]

# 여기 개인투수들이 타자를 상대한 수만큼 행을 늘리고 싶은데
# 힘듦

In [13]:
list=[]
for i in pitcher_p_data['PA']:
    for j in range(i):
        for k in range(len(pitcher_p_data)):
            list.append(pitcher_p_data.iloc[k]) #K행을j 만큼 반복해서 넣고, 

KeyboardInterrupt: 

# 팀타자 데이터에서 타점과 팀안타간의 비율 타점과 팀볼넷의 비율 구해서 타점과 상관관계 확인?

In [14]:
#팀타자 데이터들에서 년도별 팀 타율을 확인하기 위해 년도 데이터 프레임 만들어서 append
year=hitter_data['GDAY_DS'].astype(str).str.slice(start=0, stop=4).astype(int)
year.rename(columns={'GDAY_DS': 'year'}, inplace=True)
for_team_hit = pd.DataFrame(hitter_data) # for_team_hit완성
for_team_hit = pd.concat([hitter_data, year], axis=1)
for_team_hit.rename(columns={for_team_hit.columns[28]:'year'}, inplace=True)

In [15]:
year_team_hit_avg=for_team_hit.groupby(['year', 'T_ID'])["HIT"].sum()/for_team_hit.groupby(['year', 'T_ID'])["AB"].sum() #연도별 팀 타율
make_score_ratio=for_team_hit.groupby(['year', 'T_ID'])['P_HIT_CN'].sum()/for_team_hit.groupby(['year', 'T_ID'])['P_AB_CN'].sum()#득점권 타수와 득점권hit를 나누어 주어 득점권에서 안타를 칠 확률을 구한다.

In [16]:
for_team_hit=pd.DataFrame(year_team_hit_avg)
for_team_hit = pd.concat([year_team_hit_avg, make_score_ratio],axis=1) # 연도별 팀타율, 팀별 득점권에서 득점할 확률
for_team_hit.columns = ["year_team_hit_avg","make_score_ratio"]
for_team_hit

year_team_hit_avg  make_score_ratio
year T_ID                                     
2016 HH             0.289293          0.300853
     HT             0.285857          0.287747
     KT             0.275841          0.277904
     LG             0.289844          0.285511
     LT             0.287742          0.285017
     NC             0.290993          0.302391
     OB             0.298176          0.305457
     SK             0.291459          0.275727
     SS             0.292951          0.298222
     WO             0.293387          0.306429
2017 HH             0.287276          0.290100
     HT             0.302217          0.323720
     KT             0.275471          0.290040
     LG             0.281149          0.295161
     LT             0.285342          0.279820
     NC             0.293168          0.292135
     OB             0.293806          0.295754
     SK             0.271472          0.272881
     SS             0.278508          0.284639
     WO             0.290114          0.289971
2018 HH             0.275342          0.283346
     HT             0.295274          0.302023
     KT             0.274970          0.273387
     LG             0.293294          0.306657
     LT             0.289279          0.289609
     NC             0.261491          0.267997
     OB             0.309312          0.317308
     SK             0.281050          0.287758
     SS             0.287923          0.281938
     WO             0.288076          0.296062
2019 HH             0.256043          0.254725
     HT             0.263849          0.255778
     KT             0.276939          0.270084
     LG             0.267045          0.265193
     LT             0.250254          0.250000
     NC             0.278382          0.266249
     OB             0.277631          0.276692
     SK             0.262248          0.278092
     SS             0.255857          0.254112
     WO             0.281507          0.300000
2020 HH             0.239796          0.232604
     HT             0.275632          0.282288
     KT             0.290933          0.291935
     LG             0.279059          0.318099
     LT             0.274238          0.274476
     NC             0.290121          0.328836
     OB             0.302407          0.322086
     SK             0.242661          0.247379
     SS             0.274465          0.303448
     WO             0.270916          0.296358

# 위 데이터 team_data으로 합칠 필요가 있다

In [17]:
hitter_data["AVG"] = hitter_data["HIT"] / hitter_data["AB"]
all_data = hitter_data.drop(["G_ID",'HEADER_NO', 'TB_SC', 'PA', 'AB',
       'RBI', 'RUN', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'ERR', 'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN'],axis=1) 
#팀 타자 데이터에서 타율(AVG)와 게임데이, 팀 ID 제외하고 다 날림 all_data

In [18]:
all_data # 팀타자 데이터 타율, 게임데이 팀 ID만 들어간것

GDAY_DS T_ID VS_T_ID       AVG
0    20160401   LG      HH  0.214286
1    20160401   HH      LG  0.282609
2    20160401   NC      HT  0.300000
3    20160401   HT      NC  0.235294
4    20160401   SK      KT  0.277778
..        ...  ...     ...       ...
635  20200719   LT      SS  0.206897
636  20200719   HT      OB  0.250000
637  20200719   OB      HT  0.297297
638  20200719   SK      WO  0.200000
639  20200719   WO      SK  0.212121

[6400 rows x 4 columns]

In [19]:
four_ball = team_data.groupby(["T_ID","GDAY_DS"])["four_ball"].mean() #팀과 게임 날짜를 기준으로 선수들의 4구개수의 평균을 구함
steal = team_data.groupby(["T_ID","GDAY_DS"])["steal"].mean() 
RBI = team_data.groupby(["T_ID","GDAY_DS"])["RBI"].mean() 

all_data = pd.merge(all_data,four_ball, on=["T_ID","GDAY_DS"], how="left")
all_data = pd.merge(all_data,steal, on=["T_ID","GDAY_DS"], how="left")
all_data = pd.merge(all_data,RBI, on=["T_ID","GDAY_DS"], how="left")
all_data

GDAY_DS T_ID VS_T_ID       AVG  four_ball     steal       RBI
0     20160401   LG      HH  0.214286   0.073353  0.575814  0.110660
1     20160401   HH      LG  0.282609   0.078035  0.536304  0.106467
2     20160401   NC      HT  0.300000   0.080877  0.621479  0.106008
3     20160401   HT      NC  0.235294   0.077377  0.634964  0.105202
4     20160401   SK      KT  0.277778   0.070237  0.627517  0.105334
...        ...  ...     ...       ...        ...       ...       ...
6395  20200719   LT      SS  0.206897   0.080057  0.716413  0.106380
6396  20200719   HT      OB  0.250000   0.082366  0.585301  0.104298
6397  20200719   OB      HT  0.297297   0.087659  0.605671  0.120073
6398  20200719   SK      WO  0.200000   0.076455  0.548953  0.110196
6399  20200719   WO      SK  0.212121   0.087895  0.581876  0.119756

[6400 rows x 7 columns]

In [20]:
all_data[all_data.isnull().any(axis=1)] # 2016 KT선수들의 데이터가 결측치로 나와서 제거
all_data = all_data.dropna(axis=0)
all_data2 = all_data.drop(["AVG"], axis=1)

In [21]:
from sklearn.preprocessing import LabelEncoder #팀이름 라벨 인코딩
le = LabelEncoder()
all_data2["T_ID"] = le.fit_transform(all_data2["T_ID"])
all_data2["VS_T_ID"] = le.fit_transform(all_data2["VS_T_ID"])


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data2,all_data["AVG"], test_size=0.2, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(X_train,y_train)
result = rf.predict(X_test)

In [24]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,result)


0.008574909012666903

In [ ]:
#0.008596786874816291

# 팀방어율 영향을 끼칠수 있는 변수들

### 평균자책점 = 자책점 / 게임(9이닝) = 자책점 / (이닝÷9) = 자책점 X 9 / 이닝

In [52]:
#선발, 마무리로 나눠서 평균자책점 구하기 
pitcher_p_data_first=pitcher_p_data[(pitcher_p_data['START_CK'] == 1)] #선발투수평균 자책점
pitcher_p_data_first=(pitcher_p_data.groupby('P_ID')['ER'].sum()*9)/ (pitcher_p_data_first.groupby('P_ID')['INN2'].sum()/3) #선발투수 평균자책점 구하기
pitcher_p_data_closer = pitcher_p_data[(pitcher_p_data['RELIEF_CK'] == 1) | (pitcher_p_data['CG_CK'] == 1)] #중간계투, 및 마무리 평균 자책점 구하기
pitcher_p_data_closer=(pitcher_p_data_closer.groupby('P_ID')['ER'].sum()*9)/ (pitcher_p_data_closer.groupby('P_ID')['INN2'].sum()/3) 

In [67]:
team_total_team_ball = pitcher_data.groupby('T_ID')['BF'].mean()
team_total_team_ball = team_total_team_ball.mean()

In [114]:
#팀당 경기당 투구수-팀 전체 평균 투구수해서 양수면 평균보다 공을 많이 던졌고 음수면 평균보다 공을 적게 던졌다.
#공을 많이 던질수록 안타등을 맞을 확률이 높기때문에 유의미한 변수가 될수 있다.
pitcher_data_ball_count=pitcher_data['BF'] - team_total_team_ball
pitcher_data_ball_count=pd.concat([pitcher_data, pitcher_data_ball_count], axis=1)
list=[]
for i in pitcher_data.columns:
    list.append(i)
list.append('ball_count')
pitcher_data_ball_count.columns=[i for i in list]
pitcher_data_ball_count = pitcher_data_ball_count.drop(["G_ID",'HEADER_NO', 'TB_SC', 'HOLD',
       'INN2', 'PA', 'AB', 'HIT', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
       'IB', 'HP', 'KK', 'GD', 'WP', 'BK', 'ER', 'P_WHIP_RT', 'P2_WHIP_RT', 'ERR', 'R','CB_WHIP_RT'],axis=1) 
pitcher_data_ball_count


GDAY_DS T_ID VS_T_ID  CG_CK WLS   BF  ball_count
0    20160401   LG      HH      0   W  184   32.115287
1    20160401   HH      LG      0   L  196   44.115287
2    20160401   NC      HT      0   W  143   -8.884713
3    20160401   HT      NC      0   L  147   -4.884713
4    20160401   SK      KT      0   L  151   -0.884713
..        ...  ...     ...    ...  ..  ...         ...
635  20200719   LT      SS      0   W  155    3.115287
636  20200719   HT      OB      0   L  137  -14.884713
637  20200719   OB      HT      0   W  177   25.115287
638  20200719   SK      WO      0   W  134  -17.884713
639  20200719   WO      SK      0   L  173   21.115287

[6400 rows x 7 columns]

In [153]:
# 득점권 whip을 이용하여 개별 투수의 상대팀당 득점권에서 안타 볼넷 허용률의 평균을구한다
pitcher_p_whip = pitcher_p_data.groupby(['VS_T_ID', 'P_ID'])['P_WHIP_RT'].mean()
pitcher_p_whip

VS_T_ID  P_ID 
HH       50030    5.250000
         50040    0.975000
         50234    1.000000
         50404    1.363636
         50449    1.500000
                    ...   
WO       97336    0.562500
         99314    4.500000
         99445    0.763158
         99563    0.107143
         99737    1.312500
Name: P_WHIP_RT, Length: 3710, dtype: float64

In [158]:
# 자책점의 합을 이용하여 개별 투수의 상대팀당 득점권 WHIP의 평균과 상관관계를 찾아본다
pitcher_p_ER = pitcher_p_data.groupby(['VS_T_ID', 'P_ID'])['ER'].sum()
pitcher_p_ER

pitcher_p_whip=pd.DataFrame(pitcher_p_whip)
pitcher_p_ER=pd.DataFrame(pitcher_p_ER)
corr_pitcher_ER_whip = pd.merge(pitcher_p_whip, pitcher_p_ER, on="P_ID",how="left")

corr_pitcher_ER_whip.corr()

P_WHIP_RT        ER
P_WHIP_RT   1.000000  0.057241
ER          0.057241  1.000000

# 팀승률에 영향을 끼칠 수 있는 변수들

In [ ]:
Home_away_score=for_team_hit.groupby(['year','T_ID', 'TB_SC'])["RUN"].mean() # 년도별 팀마다 초공격일때 득점과 말공격일떄 득점평균
# 팀별 투수, 타자 실책의 갯수 합쳐서 팀마다 경기당 실책이 나올 확률